In [1]:
!pip install pandas scikit-learn transformers torch tensorflow -q

Carrega as bibliotecas para a execução dos modelos.

In [2]:
import numpy as np
import pandas as pd
import pickle
import torch
import nltk
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from transformers import AutoTokenizer, AutoModel
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from google.colab import files

Carrega o arquivo que será utilizado para o treinamento.

In [3]:
filename = 'transacoes_completas.csv'

if not os.path.exists(filename):
    print(f"File '{filename}' not found. Please upload the file.")
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        print(f"File '{filename}' uploaded successfully.")
    else:
        print("No file was uploaded.")
else:
    print(f"File '{filename}' is already loaded.")

File 'transacoes_completas.csv' is already loaded.


Carregar dataset do arquivo uploaded usando on_bad_lines='skip' para pular linhas com problemas de parsing, após carregado exibe as dez primeiras linhas como amostra.

In [4]:
dataset = pd.read_csv(filename, delimiter=';', on_bad_lines='skip')
print(dataset.head())

         data                 descricao categoria   Valor
0  2022-08-21  Farmácia Drogaria Araujo       SAU  113.98
1  2021-08-17           deb autor abril       ASS  156.93
2  2024-12-29  Assinatura Deezer Mensal       ASS  103.29
3  2023-09-04    lanchonete e churrasca       BAR  877.90
4  2021-04-27         rshop-sacolao che       MER  485.60


***Pré processamento do texto:***

- Baixa a lista de palavras irrelevantes (stopwords) da linguagem natural — como “de”, “o”, “e”, “para”, etc. — que geralmente não carregam muito significado em tarefas de processamento de texto.

- Cria um conjunto com todas as stopwords em português, para facilitar a checagem rápida (sets são mais eficientes que listas para esse tipo de operação).

In [5]:
nltk_data_dir = os.path.join(os.path.expanduser("~"), "nltk_data", "corpora", "stopwords")

# Verifica se o diretório de stopwords existe
if not os.path.exists(nltk_data_dir):
    nltk.download("stopwords", quiet=True)

stop_words = set(stopwords.words('portuguese'))

In [6]:
# Defina seu dicionário customizado de palavras a serem removidas
custom_stop_words = set([
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'
])

Para cada linha da coluna "descricao":
* Divide o texto em palavras (x.split()).
* Filtra aquelas que não estão nas stopwords (if word.lower() not in stop_words).
* Junta as palavras de volta em um único texto com " ".join(...).
* Salva o resultado na nova coluna chamada descricao_processed.

Exemplo prático:
Se descricao = "Conta de luz de fevereiro", após esse processamento, o texto vira algo como: "Conta luz fevereiro"
(Remove "de", que é uma stopword.)

In [7]:
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        # Combina o NLTK stopwords e a lista customizada
        all_stop_words = stop_words.union(custom_stop_words)
        tokens = [word for word in text.split() if word not in all_stop_words and len(word) > 2]
        return " ".join(tokens)
    else:
        return ""

dataset['descricao_processed'] = dataset['descricao'].apply(preprocess_text)
print(dataset[['descricao', 'descricao_processed']].head())

                  descricao       descricao_processed
0  Farmácia Drogaria Araujo  farmácia drogaria araujo
1           deb autor abril           deb autor abril
2  Assinatura Deezer Mensal  assinatura deezer mensal
3    lanchonete e churrasca      lanchonete churrasca
4         rshop-sacolao che         rshop-sacolao che


***Tokenização:***

Transforma os textos em representações numéricas vetoriais usando a técnica TF-IDF (Term Frequency–Inverse Document Frequency).

Inicializa o vetor com:
- max_features=1000: limita o vocabulário a 1000 palavras mais relevantes (com maior valor TF-IDF).
- Outros parâmetros possíveis: min_df, max_df, ngram_range, etc.

Transforma a coluna descricao_processed (que já teve stopwords removidas, por exemplo) em uma matriz esparsa de TF-IDF.
- O resultado é uma matriz com dimensões = (número de documentos, número de palavras selecionadas).
Exibe o tamanho da matriz gerada (por exemplo, 1000 documentos x 1000 palavras).

Mostra as palavras selecionadas como colunas da matriz — ou seja, o vocabulário final.

In [8]:
tfidf_vectorizer = TfidfVectorizer(min_df=10, max_df=0.80, max_features=1000, strip_accents='unicode')
tfidf_matrix = tfidf_vectorizer.fit_transform(dataset['descricao_processed'])

print("Forma da matriz TF-IDF:", tfidf_matrix.shape)
print("Features extraídas:", tfidf_vectorizer.get_feature_names_out())

Forma da matriz TF-IDF: (2000, 86)
Features extraídas: ['agro' 'agua' 'amazon' 'american' 'americana' 'assinatura' 'atac'
 'atacadao' 'aut' 'auto' 'autor' 'bar' 'bki' 'cartao' 'casa' 'central'
 'cobasi' 'com' 'comercio' 'compra' 'consulta' 'conta' 'corrente' 'cpfl'
 'credito' 'ct' 'cxe' 'dae' 'deb' 'debito' 'doc' 'drogaria' 'emporio'
 'estacioname' 'estacionamento' 'farma' 'farmacia' 'fatura' 'feira'
 'hotel' 'ifd' 'ifood' 'int' 'ipva' 'itau' 'juros' 'lanches' 'loja'
 'lojas' 'mensal' 'mercado' 'mercadolivre' 'multa' 'netflix' 'padaria'
 'pag' 'pagamento' 'parar' 'paulista' 'pay' 'pending' 'pet' 'pizza'
 'pizzaria' 'posto' 'premium' 'prime' 'restaurante' 'rotativo' 'rshop'
 'sabesp' 'sao' 'saude' 'seguro' 'shop' 'shopping' 'sisdeb' 'sp' 'super'
 'supermercado' 'tar' 'ted' 'trip' 'uber' 'vida' 'visa']


Divide o dataset em dados de treino e teste:
- X: são os textos (descrições das contas).
- y: são as classes/etiquetas (tipo da conta).
- test_size=0.2: 20% dos dados serão usados para teste.
- random_state: garante reprodutibilidade.


In [9]:
y = dataset['categoria']
X = tfidf_matrix
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Forma do conjunto de treino (features):", X_train.shape)
print("Forma do conjunto de teste (features):", X_test.shape)
print("Forma do conjunto de treino (target):", y_train.shape)
print("Forma do conjunto de teste (target):", y_test.shape)

Forma do conjunto de treino (features): (1600, 86)
Forma do conjunto de teste (features): (400, 86)
Forma do conjunto de treino (target): (1600,)
Forma do conjunto de teste (target): (400,)


#***Modelo de Regressão Logística:***
- Cria um modelo de regressão logística com até 1000 iterações permitidas no processo de ajuste.


In [10]:
model_lr = LogisticRegression(max_iter=1000, solver='saga', C=1.0, random_state=42)
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

print("\nRelatório de Classificação (Regressão Logística):")
print(classification_report(y_test, y_pred_lr))
print("\nAcurácia do modelo (Regressão Logística):", accuracy_score(y_test, y_pred_lr))


Relatório de Classificação (Regressão Logística):
              precision    recall  f1-score   support

         ABS       1.00      0.83      0.91        36
         AGU       1.00      0.94      0.97        16
         APL       1.00      0.20      0.33         5
         ASS       0.72      0.78      0.75        23
         BAR       0.38      0.67      0.48        57
         CAS       0.00      0.00      0.00         8
         CRD       0.75      0.75      0.75         4
         EDU       0.00      0.00      0.00         4
         IMP       1.00      0.29      0.44         7
         JUR       1.00      1.00      1.00         6
         LAZ       1.00      0.38      0.55         8
         LOJ       0.37      0.49      0.42        77
         MAN       0.00      0.00      0.00         5
         MER       0.73      0.61      0.67        59
         PED       1.00      0.88      0.93         8
         PET       0.71      0.83      0.77         6
         SAL       0.00      0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [11]:
import pickle
import os

if not os.path.exists('models'):
    os.makedirs('models')

with open('models/logistic_regression_model.pkl', 'wb') as f:
    pickle.dump(model_lr, f)

with open('models/logistic_regression_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

#***Modelo RamdomForest:***

Cria um pipeline com apenas uma etapa: o classificador Random Forest.
- n_estimators=100: o modelo vai construir 100 árvores para tomar decisões.
- random_state = 42: garante reprodutibilidade.

Obs: O pipeline não inclui o TF-IDF, pois os dados (X_train) já estão vetorizados.

- O modelo é treinado com os vetores de treino (X_train) e os rótulos (y_train).
- Depois, faz predições sobre os dados de teste (X_test), gerando y_pred_rf.

In [12]:
y = dataset['categoria']
X = tfidf_matrix
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline_rf = Pipeline([
    ("clf", RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_rf.fit(X_train_rf, y_train_rf)
y_pred_rf = pipeline_rf.predict(X_test_rf)

print("TF-IDF + Random Forest")
print(classification_report(y_test, y_pred_rf))
print("\nAcurácia do modelo (RamdomForest):", accuracy_score(y_test_rf, y_pred_lr))

TF-IDF + Random Forest
              precision    recall  f1-score   support

         ABS       1.00      0.89      0.94        36
         AGU       0.94      0.94      0.94        16
         APL       1.00      0.20      0.33         5
         ASS       0.82      0.78      0.80        23
         BAR       0.83      0.33      0.47        57
         CAS       0.00      0.00      0.00         8
         CRD       1.00      0.75      0.86         4
         EDU       0.00      0.00      0.00         4
         IMP       1.00      0.29      0.44         7
         JUR       1.00      1.00      1.00         6
         LAZ       1.00      0.38      0.55         8
         LOJ       0.39      0.92      0.55        77
         MAN       0.00      0.00      0.00         5
         MER       0.71      0.58      0.64        59
         PED       1.00      0.88      0.93         8
         PET       0.62      0.83      0.71         6
         SAL       0.33      0.17      0.22         6
    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [13]:
import os
import pickle

if not os.path.exists('models'):
    os.makedirs('models')

with open('models/random_forest_model.pkl', 'wb') as f:
    pickle.dump(pipeline_rf, f)

with open('models/random_forest_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

#***Método Naive Bayes:***

Implementa uma classificação de texto usando o método Bag-of-Words (BoW) com o classificador Naive Bayes (MultinomialNB).

cria o pipeline com duas etapas:
- CountVectorizer() -
Transforma os textos em vetores de contagem (BoW), ou seja, cada texto vira uma matriz onde cada elemento representa quantas vezes uma palavra aparece.
- MultinomialNB() -
Um classificador baseado em probabilidades para dados de contagem como o BoW.

Treina o modelo com os dados de treino. Ele aprende quais palavras são mais associadas a quais categorias.


In [14]:
X = dataset['descricao']
y = dataset['categoria']
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline_bow_nb = Pipeline([
    ("bow", CountVectorizer()),
    ("clf", MultinomialNB())
])

pipeline_bow_nb.fit(X_train_bow, y_train_bow)
y_pred_nb = pipeline_bow_nb.predict(X_test_bow)

print("Bag-of-Words + Naive Bayes")
print(classification_report(y_test_bow, y_pred_nb))
print("\nAcurácia do modelo (Naive Bayes):", accuracy_score(y_test_bow, y_pred_nb))

Bag-of-Words + Naive Bayes
              precision    recall  f1-score   support

         ABS       1.00      0.94      0.97        36
         AGU       0.80      1.00      0.89        16
         APL       1.00      0.60      0.75         5
         ASS       0.73      0.96      0.83        23
         BAR       0.71      0.72      0.71        57
         CAS       1.00      0.38      0.55         8
         CRD       1.00      1.00      1.00         4
         EDU       1.00      0.25      0.40         4
         IMP       1.00      0.57      0.73         7
         JUR       1.00      0.83      0.91         6
         LAZ       1.00      1.00      1.00         8
         LOJ       0.63      0.83      0.72        77
         MAN       0.00      0.00      0.00         5
         MER       0.84      0.81      0.83        59
         PED       1.00      0.75      0.86         8
         PET       1.00      0.50      0.67         6
         SAL       1.00      0.17      0.29         6


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [15]:
import pickle
import os

if not os.path.exists('models'):
    os.makedirs('models')

with open('models/naive_bayes_model.pkl', 'wb') as f:
    pickle.dump(pipeline_bow_nb, f)

with open('models/naive_bayes_vectorizer.pkl', 'wb') as f:
    pickle.dump(pipeline_bow_nb.named_steps['bow'], f)

#***Modelo BERTimbau:***

classifica textos em português usando o modelo BERTimbau da NeuralMind para transformar os textos em vetores numéricos (embeddings), e depois treina um classificador simples de regressão logística.

- Carrega o tokenizador e o modelo BERTimbau já treinado em português.


In [16]:
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


- Transforma um texto em um vetor numérico denso.
- Usa o [CLS] token (output.last_hidden_state[:, 0, :]) como representação do texto inteiro.

In [17]:
def gerar_embedding(texto):
    tokens = tokenizer(texto, return_tensors="pt", truncation=True, padding=True, max_length=32)
    with torch.no_grad():
        output = model(**tokens)
    return output.last_hidden_state[:, 0, :].squeeze().numpy()

- Seleciona um subconjunto de 1000 descrições e categorias aleatórias para não pesar demais na geração dos embeddings.

In [18]:
X_small = dataset['descricao_processed'].sample(1000, random_state=42).reset_index(drop=True)
y_small = dataset["categoria"].sample(1000, random_state=42).reset_index(drop=True)

- Para cada descrição, cria seu embedding BERT correspondente.
- Divide os embeddings e rótulos em treino e teste (70/30).

In [19]:
X_emb = np.array([gerar_embedding(t) for t in X_small])
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X_emb, y_small, test_size=0.3, random_state=42)

- Treina uma regressão logística sobre os embeddings.

In [20]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

modelo_logreg = LogisticRegression(max_iter=1000)
modelo_logreg.fit(X_train_e, y_train_e)

y_pred_logreg_e = modelo_logreg.predict(X_test_e)

print("BERTimbau + Regressão Logística")
print(classification_report(y_test_e, y_pred_logreg_e))
print("\nAcurácia do modelo (BERTimbau + Regressão Logística):", accuracy_score(y_test_e, y_pred_logreg_e))

BERTimbau + Regressão Logística
              precision    recall  f1-score   support

         ABS       0.95      0.81      0.88        26
         AGU       0.83      1.00      0.91        10
         APL       0.60      1.00      0.75         3
         ASS       0.88      0.79      0.83        19
         BAR       0.67      0.74      0.71        39
         CAS       1.00      1.00      1.00         3
         CRD       1.00      0.91      0.95        11
         EDU       1.00      1.00      1.00         3
         IMP       1.00      0.25      0.40         4
         JUR       1.00      1.00      1.00        10
         LAZ       0.91      0.77      0.83        13
         LOJ       0.73      0.82      0.77        50
         MAN       1.00      0.33      0.50         3
         MER       0.68      0.76      0.72        45
         PED       0.86      0.86      0.86         7
         PET       1.00      1.00      1.00         4
         SAL       1.00      1.00      1.00      

In [21]:
import os
import pickle
from transformers import AutoModel, AutoTokenizer

if not os.path.exists('models/bertimbau_model'):
    os.makedirs('models/bertimbau_model')

if not os.path.exists('models/bertimbau_tokenizer'):
    os.makedirs('models/bertimbau_tokenizer')

if not os.path.exists('models/bertimbau_logreg'):
    os.makedirs('models/bertimbau_logreg')

bert_model_to_save = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")
bert_tokenizer_to_save = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

bert_model_to_save.save_pretrained('models/bertimbau_model')

bert_tokenizer_to_save.save_pretrained('models/bertimbau_tokenizer')

with open('models/bertimbau_logreg/modelo_logreg.pkl', 'wb') as f:
    pickle.dump(modelo_logreg, f)

print("Modelo BERTimbau, tokenizer e modelo de regressão logística salvos na pasta 'models'!")

Modelo BERTimbau, tokenizer e modelo de regressão logística salvos na pasta 'models'!


#***Modelo CNN***

Este trecho de código está realizando a tokenização e o preenchimento (padding) de texto, para a preparação de dados de texto para redes neurais como CNNs.

Preprocessamento:
- tokenizer = Tokenizer(): Inicializa um objeto Tokenizer do Keras.
 * num_words=1000: Limita o tokenizador a considerar apenas as 1000 palavras mais frequentes no conjunto de dados.
 * oov_token="<OOV>": Define um token especial (<OOV> para Out-Of-Vocabulary, ou Fora do Vocabulário) para representar palavras que não estão entre as 1000 mais frequentes.
- tokenizer.fit_on_texts(): Treina o tokenizador nos dados de texto na coluna 'descricao_processed' do seu DataFrame. O tokenizador constrói um vocabulário com base nas palavras mais frequentes e atribui um número inteiro único a cada palavra.
- sequences = tokenizer.texts_to_sequences(): Converte as descrições de texto em sequências de números inteiros. Cada palavra em uma descrição é substituída pelo seu ID inteiro correspondente do vocabulário do tokenizador.
- padded = pad_sequences(sequences, maxlen=10, padding='post'): Garante que todas as sequências de números inteiros tenham o mesmo comprimento.
 * sequences: lista de entrada de sequências de números inteiros.
 * maxlen=10: Define o comprimento alvo para todas as sequências como 10.
 * padding='post': Adiciona preenchimento (zeros) no final das sequências com menos de 10 elementos. Se uma sequência for maior que 10, ela será truncada.



In [22]:
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['descricao_processed'])
sequences = tokenizer.texts_to_sequences(dataset['descricao_processed'])
padded = pad_sequences(sequences, maxlen=10, padding='post')

Codificação das categorias:

In [23]:
encoder = LabelEncoder()
labels = encoder.fit_transform(dataset['categoria'])
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(padded, labels, test_size=0.3)

Processamento:

Cria um modelo sequencial — onde as camadas são adicionadas uma após a outra.
- Camada de embedding: transforma cada palavra (representada como um número inteiro) em um vetor denso.
 - input_dim=1000: o vocabulário tem até 1000 palavras únicas.
 - output_dim=50: cada palavra será representada por um vetor de 50 dimensões.
- Camada convolucional 1D: desliza um "filtro" sobre a sequência de embeddings, detectando padrões locais (por exemplo: "atraso pagamento" ou "conta vencida").
 - filters=64: número de filtros aplicados.
 - kernel_size=3: o tamanho do “n-grama” textual capturado por cada filtro.
 - activation='softmax': introduz não linearidade.
- Camada de pooling: pega o valor máximo de cada filtro ao longo da sequência, reduzindo a dimensionalidade e mantendo as características mais relevantes.
- Camada totalmente conectada (hidden layer), com 64 neurônios e função de ativação softmax.
- Camada de saída, com uma unidade para cada classe única presente em labels.
- activation='softmax': converte os outputs em probabilidades somando 1 — ideal para classificação multiclasse.

In [24]:
model = Sequential([
    Embedding(input_dim=1000, output_dim=50),
    Conv1D(filters=64, kernel_size=3, activation='softmax'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')
])

Prepara e treina a rede neural com Keras para o problema de classificação multiclasse

In [25]:
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stop], verbose=1)

print("CNN")
# Get predicted classes
y_pred_cnn = np.argmax(model.predict(X_test), axis=-1)
print(classification_report(y_test, y_pred_cnn))
print("\nAcurácia do modelo (CNN):", accuracy_score(y_test, y_pred_cnn))

ValueError: Invalid dtype: object

In [ ]:
import pickle
import os

if not os.path.exists('models'):
    os.makedirs('models')

model.save('models/cnn_model.h5')

with open('models/cnn_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

with open('models/cnn_label_encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)


print("Modelos, vetorizadores, tokenizer e encoder salvos com sucesso!")